Following [this](https://docs.pydantic.dev/usage/model_config/)

# Model Config

Behaviour of pydantic can be controlled via the `Config` class on a model or a pydantic dataclass.


In [1]:
from pydantic import BaseModel, ValidationError


class Model(BaseModel):
    v: str

    class Config:  # NOTE.
        max_anystr_length = 10
        error_msg_templates = {
            "value_error.any_str.max_length": "max_length:{limit_value}",
        }


try:
    Model(v="x" * 20)
except ValidationError as e:
    print(e)
    """
    1 validation error for Model
    v
      max_length:10 (type=value_error.any_str.max_length; limit_value=10)
    """

1 validation error for Model
v
  max_length:10 (type=value_error.any_str.max_length; limit_value=10)


In [2]:
from pydantic import BaseModel, ValidationError, Extra


class Model(BaseModel, extra=Extra.forbid):  # NOTE: can also be set as so.
    a: str


try:
    Model(a="spam", b="oh no")
except ValidationError as e:
    print(e)
    """
    1 validation error for Model
    b
      extra fields not permitted (type=value_error.extra)
    """

1 validation error for Model
b
  extra fields not permitted (type=value_error.extra)


In [4]:
# If using the *pydantic* `@dataclass` decorator:

from datetime import datetime

from pydantic import ValidationError
from pydantic.dataclasses import dataclass  # NOTE.


class MyConfig:  # NOTE.
    max_anystr_length = 10
    validate_assignment = True
    error_msg_templates = {
        "value_error.any_str.max_length": "max_length:{limit_value}",
    }


@dataclass(config=MyConfig)  # NOTE.
class User:
    id: int
    name: str = "John Doe"
    signup_ts: datetime = None  # pyright: ignore


user = User(id="42", signup_ts="2032-06-21T12:00")  # pyright: ignore
try:
    user.name = "x" * 20
except ValidationError as e:
    print(e)
    """
    1 validation error for User
    name
      max_length:10 (type=value_error.any_str.max_length; limit_value=10)
    """

1 validation error for User
name
  max_length:10 (type=value_error.any_str.max_length; limit_value=10)


* Full list of options: https://docs.pydantic.dev/usage/model_config/#options

### Change behaviour globally

If you wish to change the behaviour of pydantic globally, you can **create your own custom `BaseModel` with custom `Config` since the config is inherited**:

In [6]:
from pydantic import BaseModel as PydanticBaseModel  # NOTE.


class BaseModel(PydanticBaseModel):  # NOTE.
    class Config:
        arbitrary_types_allowed = True


class MyClass:
    """A random class"""


class Model(BaseModel):  # NOTE.
    x: MyClass

* Alias stuff skipped.

### Smart Union

By default, as explained [here](https://docs.pydantic.dev/usage/types/#unions), pydantic tries to validate
(and coerce if it can) **in the order of** the `Union`.

So sometimes you may have unexpected coerced data.

In [7]:
from typing import Union

from pydantic import BaseModel


class Foo(BaseModel):
    pass


class Bar(BaseModel):
    pass


class Model(BaseModel):
    x: Union[str, int]
    y: Union[Foo, Bar]


print(Model(x=1, y=Bar()))  # NOTE: :(
# > x='1' y=Foo()

x='1' y=Foo()


To prevent this, you can enable `Config.smart_union`.

Pydantic will then check all allowed types before even trying to coerce. 

Know that this is of course **slower**, especially if your `Union` is quite big.

In [8]:
from typing import Union

from pydantic import BaseModel


class Foo(BaseModel):
    pass


class Bar(BaseModel):
    pass


class Model(BaseModel):
    x: Union[str, int]
    y: Union[Foo, Bar]

    class Config:
        smart_union = True


print(Model(x=1, y=Bar()))
# > x=1 y=Bar()

x=1 y=Bar()


> ⚠️ Note that this option **does not support compound types** yet (e.g. differentiate `List[int]` and `List[str]`).
>
> This option will be improved further once a strict mode is added in pydantic and will probably be the default behaviour in v2!